<a href="https://colab.research.google.com/github/DARSANDEV/PythonTrading/blob/LSTM-2.0/LSTM2.0%20test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

# New Section

In [39]:
#!pip install pandas_ta==0.3.14b0
!pip install pandas_ta --upgrade
#! pip install -U git+https://github.com/twopirllc/pandas-ta.git@development
#!pip install numpy==2.1.0

In [40]:
import pandas_ta as ta
print(ta.version)

ImportError: cannot import name 'NaN' from 'numpy' (/usr/local/lib/python3.11/dist-packages/numpy/__init__.py)

In [36]:
!pip list


Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.3.0
aiohappyeyeballs                   2.6.1
aiohttp                            3.11.13
aiosignal                          1.3.2
alabaster                          1.0.0
albucore                           0.0.23
albumentations                     2.0.5
ale-py                             0.10.2
altair                             5.5.0
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.7.1
arviz                              0.20.0
astropy                            7.0.1
astropy-iers-data                  0.2025.3.10.0.29.26
astunparse                         1.6.3
atpublic                           4.1.0
attrs                              25.3.0
audioread          

In [4]:
import json
import pandas as pd
from datetime import datetime
import pytz
import websocket
from IPython.display import clear_output, display

ist = pytz.timezone('Asia/Kolkata')
error_logs = []

class DataCollector:
    def __init__(self, symbol="NSE:NIFTY", time_frame="5", period=100):
        self.socketUrl = "wss://data.tradingview.com/socket.io/websocket"
        self.symbol = symbol
        self.time_frame = time_frame
        self.period = period
        self.df = pd.DataFrame(columns=['TimeStamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
        self.ws = None

    def on_message(self, ws, message):
        """Handles incoming WebSocket messages."""
        try:
            start = message.find('"s":[')
            ends = message.find(',"ns":{')
            fdata = json.loads(message[start+4:ends])

            if isinstance(fdata, list):
                for item in fdata:
                    if 'v' in item:
                        timestamp_utc = datetime.utcfromtimestamp(item['v'][0])
                        timestamp_ist = timestamp_utc.replace(tzinfo=pytz.utc).astimezone(ist)
                        item['v'][0] = timestamp_ist.strftime('%Y-%m-%d %H:%M:%S')
                        self.df.loc[len(self.df)] = item['v']
                    else:
                        error_logs.append(f"Warning: Item does not have 'v' key: {item}")
            else:
                error_logs.append(f"Error: fdata is not a list. Type: {type(fdata)}, Value: {fdata}")

        except Exception as e:
            error_logs.append(f"Error extracting candle data: {e}")

        clear_output(wait=True)
        for error in error_logs:
            print(error)
        display(self.df)

    def on_error(self, ws, error):
        """Handles WebSocket errors."""
        print(f"WebSocket Error: {error}")

    def on_close(self, ws, close_status_code, close_msg):
        """Handles WebSocket closure."""
        print("WebSocket Closed")

    def on_open(self, ws):
        """Sends initialization messages when WebSocket is opened."""
        print("WebSocket Connection Established!")

        def create_message(func, arg):
            ms = json.dumps({"m": func, "p": arg})
            msg = f"~m~{len(ms)}~m~{ms}"
            ws.send(msg)

        session_id = "0.13918.2153_mum1-charts-26-webchart-16"
        create_message("chart_create_session", [session_id, ""])
        chart_id = '=' + json.dumps({"adjustment": "splits", "session": "regular", "symbol": self.symbol})
        create_message("resolve_symbol", [session_id, "sds_sym_1", chart_id])
        create_message("create_series", [session_id, "sds_1", "s1", "sds_sym_1", self.time_frame, self.period, ""])

    def start(self):
        """Starts WebSocket connection."""
        self.ws = websocket.WebSocketApp(self.socketUrl,
                                         on_message=self.on_message,
                                         on_error=self.on_error,
                                         on_close=self.on_close)
        self.ws.on_open = self.on_open
        self.ws.run_forever()

# Function to fetch live data
def get_live_data(symbol="NSE:NIFTY", time_frame="5", period=100):
    collector = DataCollector(symbol,time_frame,period)
    collector.start()
    return collector.df


In [21]:
import pandas_ta as ta

def add_indicators(data):
    """Adds technical indicators to the data."""
    data['5MIN_RSI'] = ta.rsi(data.Close, length=14).round(2)
    data['5MIN_EMA21'] = ta.ema(data.Close, length=21).round(2)
    data['5MIN_EMA9'] = ta.ema(data.Close, length=9).round(2)
    data['5MIN_EMA50'] = ta.ema(data.Close, length=50).round(2)

    data["ATR"] = data.ta.atr(length=14).round(2)
    data["ADX"] = data.ta.adx(length=14)["ADX_14"].round(2)
    dmi = data.ta.adx(length=14)
    data["+DI"] = dmi["DMP_14"].round(2)
    data["-DI"] = dmi["DMN_14"].round(2)

    data["Pivot"] = (data["High"] + data["Low"] + data["Close"]) / 3
    data["R1"] = (2 * data["Pivot"]) - data["Low"]
    data["R2"] = data["Pivot"] + (data["High"] - data["Low"])
    data["S1"] = (2 * data["Pivot"]) - data["High"]
    data["S2"] = data["Pivot"] - (data["High"] - data["Low"])

    data[["Pivot", "S1", "S2", "R1", "R2"]] = data[["Pivot", "S1", "S2", "R1", "R2"]].round(2)
    data.dropna(inplace=True)
    data.reset_index(inplace=True)
    return data


ImportError: cannot import name 'NaN' from 'numpy' (/usr/local/lib/python3.11/dist-packages/numpy/__init__.py)

In [7]:
import numpy as np
import tensorflow as tf
from keras.models import Model
from keras.layers import LSTM, Dense, Input, Activation
from sklearn.preprocessing import MinMaxScaler

def train_lstm_model(data):
    """Trains LSTM model and returns trained model."""
    data_set = data.iloc[:, 0:11]
    sc = MinMaxScaler(feature_range=(0, 1))
    data_set_scaled = sc.fit_transform(data_set)

    X, y = [], []
    backcandles = 30
    for i in range(backcandles, data_set_scaled.shape[0]):
        X.append(data_set_scaled[i - backcandles:i])
        y.append(data_set_scaled[i, -1])

    X, y = np.array(X), np.array(y).reshape(-1, 1)

    # Split data
    splitlimit = int(len(X) * 0.8)
    X_train, X_test = X[:splitlimit], X[splitlimit:]
    y_train, y_test = y[:splitlimit], y[splitlimit:]

    # Model definition
    lstm_input = Input(shape=(backcandles, 11), name='lstm_input')
    inputs = LSTM(150, name='first_layer')(lstm_input)
    inputs = Dense(1, name='dense_layer')(inputs)
    output = Activation('linear', name='output')(inputs)
    model = Model(inputs=lstm_input, outputs=output)

    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, batch_size=15, epochs=30, validation_split=0.1)

    return model, X_test, y_test


In [ ]:
df = get_live_data(period=1000)

In [8]:
import pandas as pd

def add_target_class(data):
    """Adds Target and TargetClass to the dataset."""
    data["Adj Close"] = data["Close"].shift(1)
    data['Target'] = data['Adj Close'] - data['Open']
    data['Target'] = data['Target'].shift(-1)

    # Add TargetClass (Binary classification: 1 if price increases, 0 otherwise)
    data['TargetClass'] = (data['Target'] > 0).astype(int)

    return data

def drop_data(data):
    """Cleans and preprocesses data by removing NaN values and unnecessary columns."""
    data = data.dropna().reset_index(drop=True)  # Ensures fresh indexing
    data = data.drop(['Close', 'TimeStamp'], axis=1, errors='ignore')  # Prevent KeyError if column is missing
    return data

def preprocess_data(data):
    """Runs the full preprocessing pipeline."""
    data = add_target_class(data)
    data = drop_data(data)
    return data


In [9]:
data = get_live_data(period=1000)

Error extracting candle data: Extra data: line 1 column 4 (char 3)
Error extracting candle data: Extra data: line 1 column 2 (char 1)
Error extracting candle data: Extra data: line 1 column 2 (char 1)
Error extracting candle data: Extra data: line 1 column 2 (char 1)
Error extracting candle data: Extra data: line 1 column 2 (char 1)
Error extracting candle data: Extra data: line 1 column 2 (char 1)
Error extracting candle data: Extra data: line 1 column 2 (char 1)
Error extracting candle data: Extra data: line 1 column 2 (char 1)


,TimeStamp,Open,High,Low,Close,Volume
0,2025-02-25 13:25:00,22577.95,22580.00,22571.25,22576.55,2076073.0
1,2025-02-25 13:30:00,22577.20,22587.70,22575.45,22582.55,1928426.0
2,2025-02-25 13:35:00,22584.50,22588.85,22578.35,22585.05,1860870.0
3,2025-02-25 13:40:00,22585.25,22590.90,22582.90,22586.15,2101944.0
4,2025-02-25 13:45:00,22587.15,22595.20,22584.05,22589.70,2057586.0
...,...,...,...,...,...,...
995,2025-03-18 15:05:00,22814.40,22835.90,22810.70,22834.40,7617523.0
996,2025-03-18 15:10:00,22836.35,22845.95,22830.25,22842.95,8144817.0
997,2025-03-18 15:15:00,22842.10,22849.70,22833.95,22849.35,8319811.0
998,2025-03-18 15:20:00,22848.65,22857.25,22848.05,22855.80,7032475.0


WebSocket Closed


In [10]:
#df = get_live_data(period=100)
df=data.copy()
# Step 2: Add indicators
df = add_indicators(df)

df = preprocess_data(df)

NameError: name 'add_indicators' is not defined

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Model.load_model('/content/drive/MyDrive/Colab/lstm_model.h5')

In [2]:
# from data_collector import get_live_data
# from indicators import add_indicators
# from lstm_model import train_lstm_model
import matplotlib.pyplot as plt



# Step 3: Train LSTM model
model, X_test, y_test = train_lstm_model(df)



NameError: name 'train_lstm_model' is not defined

In [ ]:
model.save('/content/drive/MyDrive/Colab/lstm_model.h5')
# Step 4: Make predictions
y_pred = model.predict(X_test)

# Step 5: Plot results
plt.figure(figsize=(16, 8))
plt.plot(y_test, color='black', label='Test')
plt.plot(y_pred, color='green', label='Prediction')
plt.legend()
plt.show()